In [1]:
import qcportal as ptl
from openeye import oechem
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff import ForceField
import cmiles
import time
import collections

client = ptl.FractalClient()
torsion_datasets = client.list_collections("TorsionDriveDataset")
print(torsion_datasets)
#datasets = []
#for i in range(len(torsion_datasets)):
#     datasets.append(torsion_datasets.index[i][1])
#datasets = ['OpenFF Group1 Torsions']
datasets = ['OpenFF Substituted Phenyl Set 1']


                                                                                                                  tagline
collection          name                                                                                                 
TorsionDriveDataset Fragment Stability Benchmark                                                                     None
                    Fragmenter paper                                                                                 None
                    OpenFF DANCE 1 eMolecules t142 v1.0                                                              None
                    OpenFF Fragmenter Validation 1.0                                                                 None
                    OpenFF Full TorsionDrive Benchmark 1                                                             None
                    OpenFF Gen 2 Torsion Set 1 Roche                                                                 None
                    Open

In [9]:
def get_coverage_report(datasets, tid):
    for dataset_name in datasets:
        count = 0
        ds = client.get_collection("TorsionDriveDataset", dataset_name)
        ds.status("default", status="COMPLETE")
        
        params = []
        for index in ds.df.index:
            # get the dihedral indices
            dihedral_indices = ds.df.loc[index].default.keywords.dihedrals[0]
            # build the molecule from the mapped smiles 
            molecule = Molecule.from_qcschema(ds.get_entry(index))
            topology = Topology.from_molecules([molecule])

            # Let's label using the Parsley force field
            forcefield = ForceField('/Users/jessica/Documents/Grad_research/WBO_Torsions_ChayaPaper/release_1.3.0_2/fb-fit/fb-fit0/forcefield/test.offxml', allow_cosmetic_attributes=True)

            # Run the molecule labeling
            molecule_force_list = forcefield.label_molecules(topology)

            # Print out a formatted description of the torsion parameters applied to this molecule
            for mol_idx, mol_forces in enumerate(molecule_force_list):
                for force_tag, force_dict in mol_forces.items():
                    if force_tag == 'ProperTorsions':
                        for (atom_indices, parameter) in force_dict.items():
                            if(parameter.id == tid):
                                if(atom_indices == dihedral_indices):
                                    count += 1
                                    params.append(parameter.id)
                                    print(index)
                                
        counter = collections.Counter(params)
        print(dataset_name, counter)
        print(" ")
    return counter

In [15]:
#torsion_coverage = get_coverage_report(datasets, 'TIG-fit0')

torsion_coverage = get_coverage_report(['OpenFF Rowley Biaryl v1.0'], 'TIG-fit0')


[H]c1c(c([n:4][c:3](c1[H])[c:2]2[c:1](c(c(nc2[H])[H])[H])[H])[H])[H]
[H]c1c([c:1]([c:2](c(c1[H])[H])[c:3]2[n:4]c(nc(n2)[H])[H])[H])[H]
[H]c1c([n:1][c:2](nc1[H])[c:3]2[n:4]c(c(c(n2)[H])[H])[H])[H]
[H]c1c([n:1][c:2](nc1[H])[C:3]2=C(C(=C([S:4]2)[H])[H])[H])[H]
[H]c1c([n:1][c:2](nc1[H])[C:3]2=C(C(=C([N:4]2[H])[H])[H])[H])[H]
[H]c1c([n:1][c:2](nc1[H])[C:3]2=C(C(=C([O:4]2)[H])[H])[H])[H]
[H]C1=C(C(=O)[N:1]([C:2](=C1[H])[C:3]2=C(C(=C([N:4]2C([H])([H])[H])[H])[H])[H])[H])[H]
[H]c1c([c:1]([c:2](c(c1[H])[H])[C:3]2=C(N(c3[c:4]2c(c(c(c3[H])[H])[H])[H])[H])[H])[H])[H]
[H]c1c([c:1]([c:2](c(c1[H])[H])[C:3]2=[N:4]c3c(c(c(c(c3N2[H])[H])[H])[H])[H])[H])[H]
[H]c1c([c:1]([c:2](c(c1[H])[H])[C:3]2=C(C(=C([N:4]2C([H])([H])[H])[H])[H])[H])[H])[H]
[H][c:1]1c(c2c(c([c:2]1[C:3]3=C(C(=C([O:4]3)[H])[H])[H])[H])c(nc(n2)[H])[H])[H]
[H]c1c([c:1]([c:2](c(c1[H])[H])[C:3]2=[N:4]C(=C(N2[H])[H])[H])[H])[H]
[H]c1c([c:1]([c:2](c(c1[H])[H])[C:3]2=[N:4]OC(=[N+]2[H])[N-][H])[H])[H]
[H]c1c([c:1]([c:2](c(c1[H])[H])[C:3]2=NN(C(=[